In [ ]:
from google.colab import files
files.upload()

Saving final_merged_dataset.csv to final_merged_dataset (1).csv
Buffered data was truncated after reaching the output size limit.

In [ ]:
import pandas as pd
df = pd.read_csv("final_merged_dataset.csv")
print("File Uploaded successfully")
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
text_cols = [c for c in df.columns if c.lower() in ("text","content","article","body","clean_text")]
label_cols = [c for c in df.columns if c.lower() in ("label","target","class","truth")]

if text_cols:
    TEXT_COL = text_cols[0]
else:
    TEXT_COL = df.select_dtypes(include="object").columns[0]

if label_cols:
    LABEL_COL = label_cols[0]
else:
    raise ValueError("❌ Couldn't find a label column automatically. Rename your label column to 'label' or 'target' and re-run this cell.")

print("\n📝 Using TEXT column →", TEXT_COL)
print("🏷️ Using LABEL column →", LABEL_COL)

# Quick peek
display(df.head(5))

# Check label balance
print("\n🔢 Class distribution:")
print(df[LABEL_COL].value_counts())

File Uploaded successfully
Shape: (108112, 2)
Columns: ['text', 'label']

📝 Using TEXT column → text
🏷️ Using LABEL column → label


,text,label
0,CHARLIE DANIELS WARNS LIBERALS: Knock It Off O...,0
1,EPA actions in Flint water crisis to be examin...,1
2,GRIFTER HILLARY CLINTON WAS PAID For Speeches ...,0
3,"AURORA, Colo. — It took 11 days of calling ...",1
4,The largest and busiest airport in Arkansas wo...,1



🔢 Class distribution:
label
1    55926
0    52186
Name: count, dtype: int64


In [ ]:
#light cleaning using pandas
df["clean_text"] = (df["text"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip())

print("Light Cleaning done!")
display(df.head(3))

Light Cleaning done!


,text,label,clean_text
0,CHARLIE DANIELS WARNS LIBERALS: Knock It Off O...,0,CHARLIE DANIELS WARNS LIBERALS: Knock It Off O...
1,EPA actions in Flint water crisis to be examin...,1,EPA actions in Flint water crisis to be examin...
2,GRIFTER HILLARY CLINTON WAS PAID For Speeches ...,0,GRIFTER HILLARY CLINTON WAS PAID For Speeches ...


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import joblib
X = df["clean_text"].astype(str).tolist()
Y = df["label"].tolist()

# next part will handle if there are strings or ints
le = LabelEncoder()
y_enc = le.fit_transform(Y)
print("Label classes:", le.classes_)

#spliting data
SEED = 42
X_train, X_test, Y_train, Y_test = train_test_split(
    X, y_enc, test_size=0.20, stratify=y_enc, random_state=SEED
)

print("Split Completed")
print("Train Size:", len(X_train), "Test Size:", len(X_test))
print("Train label Distribution:", np.bincount(Y_train))
print("Test label Distribution:", np.bincount(Y_test))

# Save splits for embedding purposes
joblib.dump(X_train, "x_train_text.pkl")
joblib.dump(X_test, "x_test_text.pkl")
joblib.dump(Y_train, "y_train.pkl")
joblib.dump(Y_test, "y_test.pkl")
joblib.dump(le, "label_encoder.joblib")

print("\nSaved: x_train_text.pkl, x_test_text.pkl, y_train.pkl, y_test.pkl, label_encoder.joblib")



Label classes: [0 1]
Split Completed
Train Size: 86489 Test Size: 21623
Train label Distribution: [41749 44740]
Test label Distribution: [10437 11186]

Saved: x_train_text.pkl, x_test_text.pkl, y_train.pkl, y_test.pkl, label_encoder.joblib


In [ ]:
from google.colab import files

files.download("x_train_text.pkl")
files.download("x_test_text.pkl")
files.download("y_train.pkl")
files.download("y_test.pkl")
files.download("label_encoder.joblib")

# Saving these files locally so that in the next notebook we can do the BERT embeddings

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip show sentence-transformers

Name: sentence-transformers
Version: 5.1.2
Summary: Embeddings, Retrieval, and Reranking
Home-page: https://www.SBERT.net
Author: 
Author-email: Nils Reimers <info@nils-reimers.de>, Tom Aarsen <tom.aarsen@huggingface.co>
License: Apache 2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: huggingface-hub, Pillow, scikit-learn, scipy, torch, tqdm, transformers, typing_extensions
Required-by: 


In [ ]:
import os
import numpy as np
import joblib
from pathlib import Path
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm

# Settings
MODEL_NAME = "all-mpnet-base-v2"
BATCH_SIZE = 64
TRAIN_PKL = "x_train_text.pkl"
TEST_PKL  = "x_test_text.pkl"
Y_TRAIN_PKL = "y_train.pkl"
Y_TEST_PKL  = "y_test.pkl"
EMB_TRAIN_FILE = "X_train_emb.npy"
EMB_TEST_FILE  = "X_test_emb.npy"
WRAPPER_FILE = "bert_xgb_wrapper.pkl"


In [ ]:
# Load text Splits
assert Path(TRAIN_PKL).exists(), f"{TRAIN_PKL} not found. Run the split cell first"
assert Path(TEST_PKL).exists(), f"{TEST_PKL} not found. Run the split cell first"

X_tarin = joblib.load(TRAIN_PKL)
X_test = joblib.load(TEST_PKL)
Y_train = joblib.load(Y_TRAIN_PKL) if Path(Y_TRAIN_PKL).exists() else None
Y_test = joblib.load(Y_TEST_PKL) if Path(Y_TEST_PKL).exists() else None

print(f"Loaded texts - Train: {len(X_train)}, Test: {len(X_test)}")


Loaded texts - Train: 86489, Test: 21623


In [ ]:
!nvidia-smi

Mon Nov  3 17:51:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   59C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Generate and save embeddings for train and test data
# 1. Load pretrained model
'''
print("Loading SentenceTransformer model:", MODEL_NAME)
model = SentenceTransformer(MODEL_NAME)

# 2. Helper function to generate embeddings safely in batches

from tqdm.notebook import tqdm
def embed_texts(texts, batch_size=BATCH_SIZE):
    all_embs = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding batches"):
        batch = texts[i:i+batch_size]
        emb = model.encode(batch, show_progress_bar=False, convert_to_numpy=True)
        all_embs.append(emb)
    return np.vstack(all_embs)

# 3. Compute embeddings for train and test

print("\nComputing embeddings for TRAIN set...")
X_train_emb = embed_texts(X_train)
print("✅ Train embeddings created!")

print("\nComputing embeddings for TEST set...")
X_test_emb = embed_texts(X_test)
print("✅ Test embeddings created!")

# 4. Save embeddings as .npy files

np.save(EMB_TRAIN_FILE, X_train_emb)
np.save(EMB_TEST_FILE, X_test_emb)
print("\n Saved embeddings:")
print("Train:", EMB_TRAIN_FILE)
print("Test :", EMB_TEST_FILE)

# 5. Display shapes for verification

print("\nEmbedding shapes:")
print("Train:", X_train_emb.shape)
print("Test :", X_test_emb.shape)
'''
import torch
print("Using GPU:", torch.cuda.is_available())


Using GPU: True


In [ ]:
#saving this here and we will do the bert embedding in the new notebook